# importing all the libraries used in the project
import pandas as pd
import numpy as np
from numpy import sqrt 
import datetime
import googlemaps

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.graphics.tsaplots as tsa
import statsmodels.api as sm
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.vector_ar.var_model import VAR

import pmdarima as pm
from pmdarima import model_selection
from pmdarima.utils import decomposed_plot
from pmdarima.arima import decompose
from pmdarima.arima.stationarity import ADFTest
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
# run the dataset through excel
bicycledf = pd.read_csv (r'C:\jp\first\data.csv')
# did some statistical analysis
bicycledf.head()
bicycledf.describe()
pd.unique(bicycledf.from_station_name)
plt.hist(bicycledf.year)
bicycledf.info()
count = bicycledf.isna().sum()
bicycledf.nunique()


In [ ]:
# cleaning the data
bicycledf['starttime']= pd.to_datetime(bicycledf['starttime'])
bicycledf['stoptime']= pd.to_datetime(bicycledf['stoptime'])
# make new features
bicycledf['WeekDay'] = pd.DatetimeIndex(bicycledf.starttime).weekday
bicycledf['WeekDay']=bicycledf['WeekDay'].map({0:'Weekday',1:'Weekday',2:'Weekday',3:'Weekday',4:'Weekday',5:'Weekend',6:'Weekend'})

bins = [0,4,8,12,16,20,24]
labels = ['Late Night', 'Early Morning','Morning','Noon','Evening','Night']
bicycledf['Day_Phase'] = pd.cut(bicycledf['hour'], bins=bins, labels=labels, include_lowest=True)


def Seasons(s):
    if(s == 1 or s == 2 or s == 12):
        return 'Winter'
    elif(s == 3 or s == 4 or s == 5):
        return 'Spring'
    elif(s == 6 or s == 7 or s == 8):
        return 'Summer'
    elif(s == 9 or s == 10 or s == 11):
        return 'Fall'
bicycledf['Seasons']=bicycledf.month.apply(Seasons)

In [ ]:
# Performing EDA 
# plotting bar graphs for User Type, Gender and Events
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(20,5))

ax1.bar(bicycledf.usertype, bicycledf.trip_id)
ax1.set_title('Bar graph User Type')
ax1.set_xlabel('User type')
ax1.yaxis.grid()

ax2.bar(bicycledf.gender, bicycledf.trip_id)
ax2.set_title('Bar Graph Gender')
ax2.set_xlabel('Gender')
ax2.yaxis.grid()

ax3.bar(bicycledf.events, bicycledf.trip_id)
ax3.set_title('Bar Graph Events')
ax3.set_xlabel('Events')
plt.xticks(rotation=45)
ax3.yaxis.grid()

# Analysing the data
sns.displot(data=bicycledf.tripduration, kde=True)
ax[0].set_xlabel('Trip Duration (minutes)', fontsize=12)
ax[0].set_title('Trip Duration Distribution')

# Number of trips per hour
bicycledf_sub = bicycledf.loc[:, ['tripduration', 'starttime']] # Keep only 'starttime' and 'tripduration' variables
# Index dataframe by 'datetime64' data in 'starttime' variable
bicycledf_sub.index = bicycledf_sub['starttime']
weekdays = bicycledf_sub[df_sub.index.weekday < 5]
weekends = bicycledf_sub[df_sub.index.weekday > 4]
weekdays_countsPerHr = weekdays.groupby(weekdays.index.hour).size()
weekends_countsPerHr = weekends.groupby(weekends.index.hour).size()
plt.rcParams.update({'font.size': 18, 'legend.fontsize': 20})
weekdays_countsPerHr.plot(kind = 'area', stacked = False, figsize = (10, 6), color = 'darkorange',
                          linewidth = 2, label='Weekdays')

weekends_countsPerHr.plot(kind = 'area', stacked = False, color = 'darkred',
                          linewidth = 2, label='Weekends')

plt.tick_params(axis = 'both', which = 'major', labelsize = 18)
plt.title('Number of trips / hour\n')
plt.xlabel('Time of day (hr)')
plt.ylabel('Number of trips')
legend = ax.legend(loc='upper left', frameon = False)

# Trip distribution
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
sns.countplot(x='gender', data=bicycledf, ax=ax[0])
ax[0].set_title('Trip Count vs. Gender', fontsize=16)
ax[0].set_xlabel('Gender', fontsize=12)
ax[0].set_ylabel('Count', fontsize=12)
sns.boxplot(x='gender', y='tripduration', data=bicycledf, ax=ax[1])
ax[1].set_title('Trip Duration vs. Gender', fontsize=16)
ax[1].set_xlabel('Gender', fontsize=12)
ax[1].set_ylabel('Trip Duration (minutes)', fontsize=12)
plt.show()

year_month_pivot = pd.pivot_table(data, index="year", columns="month", values="trip_id", aggfunc="count")
fig, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(year_month_pivot)
plt.title("Number of Trips per Year and per Month")


df_group_daytime=bicycledf.groupby(['WeekDay','Day_Phase','gender'],as_index=False).agg({'trip_id': 'count'})
males_daytime=df_group_daytime[df_group_daytime.gender == 'Male']
females_daytime=df_group_daytime[df_group_daytime.gender == 'Female']
r_m_dayphasecount=males_daytime['trip_id'].to_list()
r_f_dayphasecount=females_daytime['trip_id'].to_list()
df_group_daytime
categories = ['Late Night', 'Early Morning','Morning','Noon','Evening','Night']


fig = go.Figure()
fig.add_trace(go.Scatterpolar(
      r=r_m_dayphasecount,
      theta=categories,
      fill='toself',
      name='Male Cyclists'
))
fig.add_trace(go.Scatterpolar(
      r=r_f_dayphasecount,
      theta=categories,
      fill='toself',
      name='Female Cyclists'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      #range=[0, 2513200]
    )),
  showlegend=True,
    title="Cycle Trip count of both males and females during different phases of a day"
)

fig.show()

In [ ]:
# converting starttime into Date time 
bicycledf['starttime'] = pd.to_datetime(bicycledf['starttime'], errors='coerce')
# Putting starttime as Date time Index
bicycledf = bicycledf.set_index(pd.DatetimeIndex(bicycledf['starttime']))
# Transforming the data into time series with hourly basis
Hourlydata= bicycledf.resample('H').agg({'temperature': np.mean, 'from_station_name': np.count_nonzero, 'gender':np.amax,'Seasons':np.amax,'Day_Phase':np.amax, 'WeekDay':np.amax, 'events':np.amax })
Hourlydata= Hourlydata.dropna()
# Plotting the time series
fig, ax = plt.subplots()
ax.set_title('Daily Total Ridership')
ax.set_ylabel('Number of Rides')
ax.set_xlabel('Date')

daily_ridership_overall = Hourlydata['from_station_name']

daily_ridership_overall.plot(ax=ax, style = '.');
# Plotting the time series but for a few months 
Hourlydata[(Hourlydata.index > '5/31/2014') & 
                        (Hourlydata.index < '8/1/2014')]['from_station_name'].plot()

In [ ]:
ts = Hourlydata['from_station_name']
# decomposing the data
decomposed_ts = decompose(Hourlydata['from_station_name'].values, 'multiplicative', m=365)

decomposed_plot(decomposed_ts, figure_kwargs={'figsize': (16, 10)})
plt.show()
# ACF plot and PACF plot
plot_acf(ts)
plt.show()
plot_pacf(ts)
plt.show()
# ADF and KPSS test
print(adf_test(ts))
def kpss_test(timeseries):
    
    print ('Results of KPSS Test:')
    dftest = kpss(timeseries)
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

kpss(ts)

ts_t_adj = ts - ts.shift(1)
ts_t_adj = ts_t_adj.dropna()
ts_t_adj.plot()
# First Model
# Spltting the data
test_start = '2017-01-01'
Hourlydata['future'] = (Hourlydata.index >= test_start).astype('int')
Hourlydata['ride_count_log'] = Hourlydata['from_station_name'].apply(lambda x: np.log(x))
Hourlydata['future']

train_hourly = Hourlydata[Hourlydata['future'] == 0]['from_station_name']
test_hourly = Hourlydata[Hourlydata['future'] == 1]['from_station_name']
# Running our First model
hourlypred = SARIMAX(train_hourly, order=(2, 0, 1), seasonal_order=(1, 0, 1, 52)).fit(disp=False)
dffg = hourlypred.get_forecast(steps = len(test_hourly))
y_pred_df = dffg.conf_int(alpha = 0.05) 
y_pred_df["Predictions"] = hourlypred.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
test= Hourlydata[Hourlydata.index>= test_start]
y_pred_df.index = test.index
y_pred_out = y_pred_df["Predictions"]


# Checking the Results(RMSE, MSE, R^2 )
arma_rmse = np.sqrt(mean_squared_error(test_hourly.values, y_pred_df["Predictions"]))
print("RMSE: ",arma_rmse)
arma_r2 = r2_score(test_hourly.values, y_pred_df["Predictions"]))
print("R2: ",arma_r2)
arma_MSE = mean_squared_error(test_hourly.values, y_pred_df["Predictions"]))
print("MSE: ",arma_mse)

# Preparing our Second model and finding hyper parameters
# Taking log
Hourlydata['ride_count_log'] = Hourlydata['from_station_name'].apply(lambda x: np.log(x))
decomposed_ts1 = decompose(Hourlydata['ride_count_log'].values, 'multiplicative', m=365)
decomposed_plot(decomposed_ts1, figure_kwargs={'figsize': (16, 10)})
plt.show()
ts2 = Hourlydata['ride_count_log']
plot_acf(ts2)
plt.show()
plot_pacf(ts2)
plt.show()
# Differencing the data
ts2diff1 = ts2.diff()
ts2diff1 = ts2diff1.dropna()
plot_acf(ts2diff1, alpha=0.05)
plt.show()
plot_pacf(ts2diff1, alpha=0.05)
plt.show()
ts2diff2 = ts2diff1.diff()
ts2diff2 = ts2diff2.dropna()
plot_acf(ts2diff2, alpha=0.05)
plt.show()
plot_pacf(ts2diff2, alpha=0.05)
plt.show()
ts2diff3 = ts2diff2.diff()
ts2diff3 = ts2diff3.dropna()
plot_acf(ts2diff3, alpha=0.05)
plt.show()
plot_pacf(ts2diff3, alpha=0.05)
plt.show()
# Second Model
seas_mod_hourly2 = SARIMAX(train_hourly, order=(1, 3, 1), seasonal_order=(1, 3, 1, 24)).fit(disp=False)
dffg2 = seas_mod_hourly2.get_forecast(steps = len(test_hourly))
y_pred_df2 = dffg2.conf_int(alpha = 0.05) 
y_pred_df2["Predictions"] = seas_mod_hourly2.predict(start = y_pred_df2.index[0], end = y_pred_df2.index[-1])
test= Hourlydata[Hourlydata.index>= test_start]
y_pred_df2.index = test.index
y_pred_out2 = y_pred_df2["Predictions"]

# Checking the Results(RMSE, MSE, R^2 )
arma_rmse = np.sqrt(mean_squared_error(test_hourly.values, y_pred_df2["Predictions"]))
print("RMSE: ",arma_rmse)
arma_r2 = r2_score(test_hourly.values, y_pred_df2["Predictions"]))
print("R2: ",arma_r2)
arma_MSE = mean_squared_error(test_hourly.values, y_pred_df2["Predictions"]))
print("MSE: ",arma_mse)

#Preparing 3rd model
# Making Dummy Variables using cat.codes
Hourlydata["gender"] = Hourlydata["gender"].astype('category')
Hourlydata["genderNew"] = Hourlydata["gender"].cat.codes

Hourlydata["Seasons"] = Hourlydata["Seasons"].astype('category')
Hourlydata["SeasonsNew"] = Hourlydata["Seasons"].cat.codes

Hourlydata["Day_Phase"] = Hourlydata["Day_Phase"].astype('category')
Hourlydata["Day_PhaseNew"] = Hourlydata["Day_Phase"].cat.codes

Hourlydata["WeekDay"] = Hourlydata["WeekDay"].astype('category')
Hourlydata["WeekDaynew"] = Hourlydata["WeekDay"].cat.codes

Hourlydata["events"] = Hourlydata["events"].astype('category')
Hourlydata["eventsnew"] = Hourlydata["events"].cat.codes

Hourlydata.head()
# making a linear regression model
from patsy import dmatrices
test= Hourlydata[Hourlydata.index>= test_start]
train= Hourlydata[Hourlydata.index< test_start]
expr = 'from_station_name ~ temperature + genderNew + SeasonsNew + Day_PhaseNew + WeekDaynew + eventsnew'
y_train, X_train = dmatrices(expr, train, return_type='dataframe')
y_test, X_test = dmatrices(expr, test, return_type='dataframe')
# OLS fit and summary
from statsmodels.regression import linear_model
olsr_results = linear_model.OLS(y_train, X_train).fit()
olsr_results.summary()
# Analysing the new results for hyperparameters
tsa.plot_acf(olsr_results.resid, alpha=0.05)
plt.show()
olsr_results2 = olsr_results.resid.apply(lambda x: np.log(x))
olsr_resid_diff_1 = olsr_results2.diff()
olsr_resid_diff_1 = olsr_resid_diff_1.dropna()

tsa.plot_acf(olsr_resid_diff_1, alpha=0.05)
plot_pacf(olsr_resid_diff_1, alpha=0.05)
plt.show()

dftest = adfuller(olsr_resid_diff_1.dropna().values, autolag = 'AIC')

print("1. ADF : ",dftest[0])
print("2. P-Value : ", dftest[1])
print("3. Num Of Lags : ", dftest[2])
print("4. Num Of Observations Used For ADF Regression and Critical Values Calculation :", dftest[3])
print("5. Critical Values :")
for key, val in dftest[4].items():
    print("\t",key, ": ", val)
    
olsr_resid_diff_2 = olsr_resid_diff_1.diff()
olsr_resid_diff_2 = olsr_resid_diff_2.dropna()
plot_acf(olsr_resid_diff_2, alpha=0.05)
plot_pacf(olsr_resid_diff_2, alpha=0.05)
plt.show()
olsr_resid_diff_3 = olsr_resid_diff_2.diff()
olsr_resid_diff_3 = olsr_resid_diff_3.dropna()
plot_acf(olsr_resid_diff_3, alpha=0.05)
plt.show()

# Running our 4th model
X_train_minus_intercept = X_train.drop('Intercept', axis=1)
sarimax_model = ARIMA(endog=y_train, exog=X_train_minus_intercept,order=(2,1,2))
sarimax_results = sarimax_model.fit()
sarimax_results.summary()

#plot results 
sarimax_results.plot_diagnostics()

#forecasting results to compare with test
X_test_minus_intercept = X_test.drop('Intercept', axis=1)
predictions = sarimax_results.get_forecast(steps=24, exog=X_test_minus_intercept[:24])
predictions.summary_frame()

#plotting the results
predicted, = plt.plot(X_test_minus_intercept[:24].index, predictions.summary_frame()['mean'], 'go-', label='Predicted')
actual, = plt.plot(X_test_minus_intercept[:24].index, y_test[:24], 'ro-', label='Actual')
 lower, = plt.plot(X_test_minus_intercept[:24].index, predictions.summary_frame()['mean_ci_lower'], color='#990099', marker='.', linestyle=':', label='Lower 95%')
upper, = plt.plot(X_test_minus_intercept[:24].index, predictions.summary_frame()['mean_ci_upper'], color='#0000cc', marker='.', linestyle=':', label='Upper 95%')
plt.fill_between(X_test_minus_intercept[:24].index, predictions.summary_frame()['mean_ci_lower'], predictions.summary_frame()['mean_ci_upper'], color = 'b', alpha = 0.2)
plt.legend(handles=[predicted, actual, lower, upper])
plt.show()

# Making dummy variables using sklearn
ohe= OneHotEncoder(sparse= "FALSE")
Y= Newdf.from_station_id
X = Newdf.drop(["from_station_id","from_station_name"], axis="columns")
X
column_Y= make_column_transformer((OneHotEncoder(), ["gender", "Seasons", "Day_Phase", "WeekDay", "events"]), remainder= "passthrough")
column_Y.fit_transform(X)
# Making Pipeline to connect dummy variable, linear reg and ARIMA model
linreg= LinearRegression()
# Running our 5th model
pipe= make_pipeline(column_Y, linreg, pm.AutoARIMA(seasonal=True, suppress_warnings=True))
pipe.fit(train)
pipe.predict(5)

# Running our 6th model (auto arima ) and comparing models with AIC value

model = pm.auto_arima(train["from_station_name"], start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
# checking model summary
print(model.summary())
# Running our 7th model (auto arima with seasonality ) and comparing models with AIC value
model2 = pm.auto_arima(train["from_station_name"], start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=24,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=True,   # Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
# checking model summary
print(model2.summary())



In [ ]:
# Selecting 10 stations for our new models
Nearby = bicycledf.loc(axis=0)[bicycledf['from_station_name'].isin(["Blackstone Ave & Hyde Park Blvd","Cornell Ave & Hyde Park Blvd", "Ellis Ave & 55th St", "Ellis Ave & 58th St", "Ellis Ave & 60th St", "Kimbark Ave & 53rd St","Lake Park Ave & 56th St", "Museum of Science and Industry", "Shore Dr & 55th St", "Woodlawn Ave & 55th St" ])]
Stations =["Blackstone Ave & Hyde Park Blvd","Cornell Ave & Hyde Park Blvd", "Ellis Ave & 55th St", "Ellis Ave & 58th St", "Ellis Ave & 60th St", "Kimbark Ave & 53rd St","Lake Park Ave & 56th St", "Museum of Science and Industry", "Shore Dr & 55th St", "Woodlawn Ave & 55th St" ]
len(Stations)

# Creating a new time series dataset with selected stations and daily basis
Newdf=Nearby.groupby([pd.Grouper(freq='1D', key='starttime'),'from_station_name']).agg({'temperature': np.mean, 'from_station_id': np.count_nonzero, 'gender':np.amax,'Seasons':np.amax,'Day_Phase':np.amax, 'WeekDay':np.amax, 'events':np.amax }).sort_index()
Newdf = Newdf.reset_index(level=1)
Newdf

# Creating a pivot table which contains time series of each station
df_d=pd.pivot_table(Nearby[['from_station_name','starttime']],aggfunc='count',index=Nearby['starttime'].dt.date,columns=['from_station_name'],fill_value=0)  #date

df_d.index = pd.to_datetime(df_d.index)
df_d.head()

# Create new dictionary to store values
df_dict = {}

for neighborhood in Stations:
    
    df_dict[neighborhood] = Newdf

# Create new dataframe that contains the sum of ridership of ech station
df_neighborhood_sum = pd.DataFrame(index=Stations, columns=['to_station_id'])
i = 0
for df in df_dict.values():
    
    df_neighborhood_sum.iloc[i,:] = df_d.iloc[:,i].sum()
    i += 1
    
df_neighborhood_sum.sort_values('to_station_id', inplace=True)

# Plotting the sum of ridership
fig, ax = plt.subplots(figsize=(10,10))

ax.barh(y=df_neighborhood_sum.index, width=df_neighborhood_sum['to_station_id']);

# Plotting time series of each station
fig, ax = plt.subplots(nrows=len(Stations), figsize=(15,60), sharex=False)
i = 0

for  neighborhood,index in df_dict.items():
    df_d.iloc[:,i].plot(ax=ax[i])
    ax[i].set_title(neighborhood)
    i += 1

    
plt.tight_layout();

# Performing ADF test on each station
df_stationary = pd.DataFrame(index=Stations, columns=['stat_pValue'])

for i, index in enumerate(df_dict.values()):
    p = adf_test(df_d.iloc[:,i])
    
    df_stationary.iloc[i,:] = [p]
    
df_stationary

# Create new functions helping in providing results
def metricss(y_true, y_pred, print_output):
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred, squared=False)
    r2 = metrics.r2_score(y_true, y_pred)
    
    if print_output:
       
        print(f"MAE: {mae:,.4f}")
        print(f"RMSE: {mse:,.4f}")
        print(f"r^2: {r2:,.4f}")
    
    return [mae, mse, r2]

# Create another function to run the SARIMA model on each station

def run_model(results, df_preds_, neighborhood, ts, order, seasonal_order)
    
    train = ts[ts['future'] == 0]
    test = ts[ts['future'] == 1]

    sari_model = SARIMAX(train['ride_count'], order=order, seasonal_order=seasonal_order).fit(maxiter=1000, disp=False)
    
    # Add preds to predictions DataFrame
    preds = sari_model.forecast(steps = len(test))
    df_preds_.loc[:,neighborhood] = preds
    
    # Retrieve metrics
    model_results = report_metrics(test['from_station_name'], preds, False)
    
    # Calculate actual vs. predicted rides for 2021
    actual_rides_2018 = ts[ts.index > '12/31/2017']['from_station_name'].sum()
    pred_rides_2018 = df_preds_[df_preds_.index > '12/31/2017'][neighborhood].sum()
    
    ride_delta = np.abs(pred_rides_2021 - actual_rides_2021)
    
    # Add results to results dataframe
    results.loc[neighborhood, 'model'] = sari_model
    results.loc[neighborhood, 'order'] = order
    results.loc[neighborhood, 'seasonal_order'] = seasonal_order
    results.loc[neighborhood, 'MAE'] = model_results[1]
    results.loc[neighborhood, 'MSE'] = model_results[2]
    results.loc[neighborhood, 'R2'] = model_results[3]
    
    return results, df_preds_

# Running the SARIMA model
i=0
for neighborhood,index in df_dict.items():
    df_results, df_preds = run_model(df_results, df_preds, neighborhood, df_d.iloc[:,i] 
                                    (1, 1, 0), (0, 1, 1, 52), 
                                    df_results.loc[neighborhood,'ridership'], i+=1)
df_results

# Running VAR models
# Splitting the data
X_trr, X_tes = df_ddf[:int(0.8*(len(df_ddf)))], df_ddf[int(0.8*(len(df_ddf))):]

# Differencing
X_train_diff =(X_trr).diff().dropna()
X_train_diff.describe()

# ACF and PACF plots
fig, ax = plt.subplots(1,2, figsize=(10,5)) 
ax[0] = plot_acf(X_train_diff['starttime']["Blackstone Ave & Hyde Park Blvd"], ax=ax[0])
ax[1] = plot_pacf(X_train_diff['starttime']["Blackstone Ave & Hyde Park Blvd"], ax=ax[1])

# Running the model
modelnew = VAR(endog=X_train_log_diff)
res = modelnew.select_order(10)
res.summary()

model_fit = modelnew.fit(maxlags=3)
#Print a summary of the model results
model_fit.summary()

# Get the lag order
lag_order = model_fit.k_ar
print(lag_order)
# Input data for forecasting
input_data = X_train_diff.values[-lag_order:]
print(input_data)
# forecasting
pred = model_fit.forecast(y=input_data, steps=290)
pred = (pd.DataFrame(pred, index=X_tes.index, columns=[Stations]))
print(pred)

# Another VAR model
# Splitting the data
trainstations = df_d[:int(0.8*(len(df_d)))]
teststations = df_d[int(0.8*(len(df_d))):]

# Run the model
model = VAR(endog=trainstations)
model_fit = model.fit()
prediction = model_fit.forecast(model_fit.y, steps=len(teststations))

# Converting test values to data frame
testt= pd.DataFrame.from_dict(teststations)
testt["starttime"]["Blackstone Ave & Hyde Park Blvd"]

# Comaring the model with test values
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[Stations])
for j in range(0,10):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

#check rmse
for i in Stations:
    print('rmse value for', i, 'is : ', sqrt(mean_squared_error(pred[i], testt["starttime"][i])))
    
# Forecasting ridership for each station
model = VAR(endog=df_d)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=20)
print(yhat)

# Finding Distance matrix for the results

gg= Nearby.iloc[[0,1,2,3,14,40002,40016,40028,50009,50027]]
# Connecting python to google api
gmaps = googlemaps.Client(key = "API Key")

# no of stations
n_Station = 10

# creating a distance matrix with zero entries between all pairs of stations
stations_distance_matrix = np.zeros((10,10))

for i in range(n_Station):
    # station, latitude, longitude for origin
    origin_city, origin_lat, origin_lng = gg.iloc[i][["from_station_name", "latitude_start", "longitude_start"]]
    
    for j in range(i+1, n_Station):
        # city, latitude, longitude for destination
        dest_city, dest_lat, dest_lng = gg.iloc[j][["from_station_name", "latitude_start", "longitude_start"]]
        # calculate 
        stations_distance_matrix[i, j] = gmaps.distance_matrix((origin_lat, origin_lng), (dest_lat, dest_lng), 
                                                             mode="bicycling")["rows"][0]["elements"][0]["distance"]["value"]/1000